# Paper Review Notebook
This notebook reads a paper file and sends it to an AI service (OpenAI or Gemini) to generate a review.
Provide the filename of your paper, the API key, and the service base URL, then run the `review_paper` function.

In [ ]:
# Install required libraries if needed
# !pip install openai google-generativeai pdfminer.six

In [ ]:
import os
from typing import Literal
from pdfminer.high_level import extract_text as pdf_extract_text

try:
    import openai
except ImportError:
    openai = None

try:
    import google.generativeai as genai
except ImportError:
    genai = None

In [ ]:
def load_text(filename: str) -> str:
    if filename.lower().endswith('.pdf'):
        return pdf_extract_text(filename)
    with open(filename, 'r', encoding='utf-8') as f:
        return f.read()


def review_paper(filename: str, api_key: str, base_url: str | None = None, service: Literal['openai', 'gemini'] = 'openai', model: str | None = None) -> str:
    text = load_text(filename)
    if service == 'openai':
        if openai is None:
            raise ImportError('openai package is not installed')
        openai.api_key = api_key
        if base_url is not None:
            openai.api_base = base_url
        if model is None:
            model = 'gpt-4o'
        messages = [
            {"role": "system", "content": "You are an expert academic reviewer. Provide a concise review summarizing the paper, listing strengths, weaknesses, and suggestions."},
            {"role": "user", "content": text}
        ]
        response = openai.ChatCompletion.create(model=model, messages=messages)
        return response.choices[0].message.content.strip()
    elif service == 'gemini':
        if genai is None:
            raise ImportError('google-generativeai package is not installed')
        if base_url is None:
            genai.configure(api_key=api_key)
        else:
            genai.configure(api_key=api_key, api_endpoint=base_url)
        if model is None:
            model = 'gemini-pro'
        model_obj = genai.GenerativeModel(model)
        resp = model_obj.generate_content(f'Please provide a peer review for the following paper:
{text}')
        return resp.text.strip()
    else:
        raise ValueError('Unsupported service: ' + service)

In [ ]:
# Example usage
filename = 'path/to/paper.pdf'   # Update with your file
api_key = 'sk-...'               # Your API key
base_url = 'https://api.openai.com/v1'  # Example base URL

# Choose service='openai' or 'gemini'
review = review_paper(filename, api_key, base_url=base_url, service='openai')
print(review)